# Monitoring for Agent Framework

When you deploy your agent with databricks.agents.deploy() it will automatically setup a monitor associated with the given mlflow experiment. This code show how to update that monitor. 

In [0]:
# %pip install -q databricks-agents>=0.18.1 mlflow
# dbutils.library.restartPython()

In [0]:
from databricks.agents import list_deployments, get_deployments, delete_deployment
import mlflow
from src.utils import set_mlflow_experiment
from configs.project import get_project_config

In [0]:
projectConfig = get_project_config()

config_file = "../configs/rag_agent.yaml"
multi_agent_config = mlflow.models.ModelConfig(development_config="../configs/rag_agent.yaml")
experiment = set_mlflow_experiment(multi_agent_config.get("databricks_resources").get("mlflow_experiment_name"))

model_name = multi_agent_config.get("databricks_resources").get("model_name")
UC_MODEL_NAME = f"{projectConfig.uc_catalog}.{projectConfig.uc_schema}.{model_name}"
print(UC_MODEL_NAME)

In [0]:
# Get the deployment for a specific agent model name and version
agent_model_name = UC_MODEL_NAME  # Set to your Unity Catalog model name
agent_model_version = 1  # Set to your agent model version
deployment = get_deployments(model_name=agent_model_name, model_version=agent_model_version)[0]

endpoint_name = deployment.__dict__.get("endpoint_name")
print(endpoint_name)

In [0]:
from databricks.agents.monitoring import update_monitor

#update the monitor
monitor = update_monitor(
    endpoint_name = endpoint_name,
    monitoring_config = {
        "sample": 1.0,  # Sample all requests - DEMO ONLY
        "metrics": ['guideline_adherence', 'groundedness', 'safety', 'relevance_to_query'],
        "global_guidelines": {
            "english": ["The response must be in English"],
            "clarity": ["The response must be clear, coherent, and concise"],
        }
    }
)

In [0]:
display(spark.table(monitor.evaluated_traces_table).filter("evaluation_status != 'skipped'"))

In [0]:
from databricks.agents.monitoring import create_monitor

#create monitor; this is not needed if you deployed the model with agents_deploy on databricks-agents >0.18.0
# TODO: set experiment_id
if False:
    monitor = create_monitor(
        endpoint_name = endpoint_name,
        monitoring_config = {
            "sample": 1.0, 
            "metrics": ['guideline_adherence', 'groundedness', 'safety', 'relevance_to_query'],
            "global_guidelines": {
                "english": ["The response must be in English"],
                "clarity": ["The response must be clear, coherent, and concise"],
            }
        },
        experiment_id="123..." #optional config to log monitor to an experiment
    )